<font size="5">Topic Cluster: Strat Procurement</font>

In [1]:
#import packages
import re
import pandas as pd
import numpy as np
from pprint import pprint

import spacy

from nltk.stem import WordNetLemmatizer
import nltk; nltk.download('stopwords')

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ipalacio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:972: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:1186: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\t_sne.py:420: DeprecationWarning: invalid escape sequence \s
  """


In [3]:
#import data
df = pd.read_excel("L://TEAMS//pur//ANALYTICS//Projects//IPPS_CRMTopicClusters//Data Sources//TopicCluster_StratPro.xlsx")
print(df.head())

#looking at text and converting to list
df_txt = df.iloc[:, -1]
df_txt_list = df_txt.tolist()

   ASK Case: Case Number                     Primary Category  \
0                 461299  Adding/Updating Suppliers or Payees   
1                 461334            Buying Goods and Services   
2                 461460            Buying Goods and Services   
3                 461480            Buying Goods and Services   
4                 461505  Adding/Updating Suppliers or Payees   

               Sub-Category                                          Subject  \
0            In Marketplace                                Marketplace Payee   
1  Approvals on Requisition                           Contract Expired - IDT   
2     Order Delivery Status                   Dell Computer Order Cancelled?   
3                 iRequests  update supplier address in Marketplace - URGENT   
4            In Marketplace               need vendor updated in marketplace   

                                         Description  
0  Hello, \nI submitted a request for a new payee...  
1  I placed an ord

In [4]:
#Play around with how the text is read in the list
print(df_txt_list[0])

Hello, 
I submitted a request for a new payee. She is already in "mypayments" because she used to be a student in our grad program. She graduated last year in Acting and also worked for the dept. as a GSR/TA. One of our faculty members hired her to act in a project that she is working on. She hired her because she knows her acting specifically because she was trained here at the University and she fit the role she was portraying perfectly. The issue I am encountering now is that this was done prior to getting approval. The faculty did not know she needed to get this approved. The money is coming from a senate grant award. I received an email requesting I fill out forms, but all the forms  are for work that has not been done. What is the correct way to proceed? I attempted to pay her via MyPayments, but was told that this needed to be processed via Marketplace.

Payee name: Mary Glen Fredrick
Payee id: 00480556P


Thank you, 
Jessenia Linares
x46003


In [5]:
#NLTK Stopwords Prep
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
domain_stop_words = ['hi', 'hello', 'thank', 'thanks', 'com', 'the', 're', 'php', 'http', 'XXXXXXXXX']
stop_words.extend(domain_stop_words)
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
#Text pre-processing. Everything string & lowercase
df_txt_list = [str(x) for x in df_txt_list]
df_txt_list = [x.lower() for x in df_txt_list]

#Text pre-processing. Getting rid of emails, punctuation, strings that start with numbers
df_txt_list = [re.sub('\S*@\S*\s?', ' ', x) for x in df_txt_list]
df_txt_list = [re.sub('[^\w\s]', ' ', x) for x in df_txt_list]
df_txt_list = [re.sub('\s[0-9]+', '', x) for x in df_txt_list]

<>:6: DeprecationWarning: invalid escape sequence \S
<>:7: DeprecationWarning: invalid escape sequence \w
<>:8: DeprecationWarning: invalid escape sequence \s
<>:6: DeprecationWarning: invalid escape sequence \S
<>:7: DeprecationWarning: invalid escape sequence \w
<>:8: DeprecationWarning: invalid escape sequence \s
<>:6: DeprecationWarning: invalid escape sequence \S
<>:7: DeprecationWarning: invalid escape sequence \w
<>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-6-d6759a1de52c>:6: DeprecationWarning: invalid escape sequence \S
  df_txt_list = [re.sub('\S*@\S*\s?', ' ', x) for x in df_txt_list]
<ipython-input-6-d6759a1de52c>:7: DeprecationWarning: invalid escape sequence \w
  df_txt_list = [re.sub('[^\w\s]', ' ', x) for x in df_txt_list]
<ipython-input-6-d6759a1de52c>:8: DeprecationWarning: invalid escape sequence \s
  df_txt_list = [re.sub('\s[0-9]+', '', x) for x in df_txt_list]


In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df_txt_list))

print(data_words[:1])

[['hello', 'submitted', 'request', 'for', 'new', 'payee', 'she', 'is', 'already', 'in', 'mypayments', 'because', 'she', 'used', 'to', 'be', 'student', 'in', 'our', 'grad', 'program', 'she', 'graduated', 'last', 'year', 'in', 'acting', 'and', 'also', 'worked', 'for', 'the', 'dept', 'as', 'gsr', 'ta', 'one', 'of', 'our', 'faculty', 'members', 'hired', 'her', 'to', 'act', 'in', 'project', 'that', 'she', 'is', 'working', 'on', 'she', 'hired', 'her', 'because', 'she', 'knows', 'her', 'acting', 'specifically', 'because', 'she', 'was', 'trained', 'here', 'at', 'the', 'university', 'and', 'she', 'fit', 'the', 'role', 'she', 'was', 'portraying', 'perfectly', 'the', 'issue', 'am', 'encountering', 'now', 'is', 'that', 'this', 'was', 'done', 'prior', 'to', 'getting', 'approval', 'the', 'faculty', 'did', 'not', 'know', 'she', 'needed', 'to', 'get', 'this', 'approved', 'the', 'money', 'is', 'coming', 'from', 'senate', 'grant', 'award', 'received', 'an', 'email', 'requesting', 'fill', 'out', 'forms',

In [8]:
#initiate blank list
text_words = []

#for loop to tokenize all words in
for sentence in df_txt_list:
    temp = gensim.utils.simple_preprocess(sentence)
    text_words.append(temp)

In [10]:
print(text_words[:1])

[['hello', 'submitted', 'request', 'for', 'new', 'payee', 'she', 'is', 'already', 'in', 'mypayments', 'because', 'she', 'used', 'to', 'be', 'student', 'in', 'our', 'grad', 'program', 'she', 'graduated', 'last', 'year', 'in', 'acting', 'and', 'also', 'worked', 'for', 'the', 'dept', 'as', 'gsr', 'ta', 'one', 'of', 'our', 'faculty', 'members', 'hired', 'her', 'to', 'act', 'in', 'project', 'that', 'she', 'is', 'working', 'on', 'she', 'hired', 'her', 'because', 'she', 'knows', 'her', 'acting', 'specifically', 'because', 'she', 'was', 'trained', 'here', 'at', 'the', 'university', 'and', 'she', 'fit', 'the', 'role', 'she', 'was', 'portraying', 'perfectly', 'the', 'issue', 'am', 'encountering', 'now', 'is', 'that', 'this', 'was', 'done', 'prior', 'to', 'getting', 'approval', 'the', 'faculty', 'did', 'not', 'know', 'she', 'needed', 'to', 'get', 'this', 'approved', 'the', 'money', 'is', 'coming', 'from', 'senate', 'grant', 'award', 'received', 'an', 'email', 'requesting', 'fill', 'out', 'forms',

In [66]:
# Build the bigram
bigram = gensim.models.Phrases(text_words, min_count=5, threshold=100)

In [67]:
#Removeal of stop words
text_words_no_stops = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in text_words]

#Make texts into bigram model
text_words_no_stops_bigarm = [bigram[doc] for doc in text_words_no_stops]

In [68]:
print(text_words_no_stops_bigarm[12])

['would', 'like', 'know', 'long', 'take', 'get', 'response', 'po', 'need', 'purchase', 'chair', 'lab', 'submitted', 'irequest', 'juneth', 'also', 'difficult', 'able', 'find', 'contact', 'list', 'people', 'phone', 'numbers', 'call', 'follow', 'someone', 'could', 'please', 'get', 'back', 'asap', 'would', 'greatly_appreciate', 'kindly', 'angelica']


In [69]:
#Function for lemmatization due to many steps
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [70]:
#getting spacy 'en' and lemmetizing bi-grams
nlp = spacy.load('en', disable=['parser', 'ner'])
data_lemmatized = lemmatization(text_words_no_stops_bigarm, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [93]:
print(data_lemmatized[0])

['submit', 'request', 'already', 'mypayment', 'use', 'student', 'grad', 'program', 'graduate', 'last_year', 'acting', 'also', 'work', 'faculty', 'member', 'hire', 'project', 'work', 'hire', 'know', 'act', 'specifically', 'train', 'university', 'fit', 'role', 'portraying', 'perfectly', 'issue', 'encounter', 'do', 'prior', 'get', 'approval', 'faculty', 'know', 'need', 'approve', 'money', 'come', 'receive', 'email', 'request', 'fill', 'form', 'form', 'work', 'do', 'correct', 'way', 'proceed', 'attempt', 'pay', 'mypayment', 'tell', 'need', 'process', 'linare']


In [72]:
#Create Dictionary
id2word = gensim.corpora.Dictionary(data_lemmatized)

#Create Corpus
texts = data_lemmatized

#Build TDF
corpus = [id2word.doc2bow(text) for text in texts]

In [73]:
#Read Corups/TDF per case
[[(id2word[id], freq) for id, freq in cp] for cp in corpus]

[[('act', 1),
  ('acting', 1),
  ('already', 1),
  ('also', 1),
  ('approval', 1),
  ('approve', 1),
  ('attempt', 1),
  ('come', 1),
  ('correct', 1),
  ('do', 2),
  ('email', 1),
  ('encounter', 1),
  ('faculty', 2),
  ('fill', 1),
  ('fit', 1),
  ('form', 2),
  ('get', 1),
  ('grad', 1),
  ('graduate', 1),
  ('hire', 2),
  ('issue', 1),
  ('know', 2),
  ('last_year', 1),
  ('linare', 1),
  ('member', 1),
  ('money', 1),
  ('mypayment', 2),
  ('need', 2),
  ('pay', 1),
  ('perfectly', 1),
  ('portraying', 1),
  ('prior', 1),
  ('proceed', 1),
  ('process', 1),
  ('program', 1),
  ('project', 1),
  ('receive', 1),
  ('request', 2),
  ('role', 1),
  ('specifically', 1),
  ('student', 1),
  ('submit', 1),
  ('tell', 1),
  ('train', 1),
  ('university', 1),
  ('use', 1),
  ('way', 1),
  ('work', 3)],
 [('approve', 1),
  ('get', 1),
  ('know', 1),
  ('able', 1),
  ('assignee', 1),
  ('contract', 1),
  ('error', 1),
  ('expire', 1),
  ('fix', 1),
  ('let', 1),
  ('order', 2),
  ('place', 1

In [74]:
#wordfreq based on df_txt_list

# df_txt_list_v2 = [x for x in df_txt_list if not x in stop_words] 

# wordfreq = {}
# for sentence in df_txt_list_v2:
#     tokens = nltk.word_tokenize(sentence)
#     for token in tokens:
#         if token not in wordfreq.keys():
#             wordfreq[token] = 1
#         else:
#             wordfreq[token] += 1
            
# pprint(wordfreq)

In [75]:
#test = pd.DataFrame(list(wordfreq.items())).sort_values(by = 1, ascending = False)
#test.to_csv("testing.csv")

In [76]:
#Building and running model
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word = id2word,
                                           num_topics = 10, 
                                           random_state = 100,
                                           update_every = 1, 
                                           chunksize = 100,
                                           passes = 10, 
                                           alpha = 'auto',
                                           per_word_topics = True)

In [77]:
pprint(lda_model.print_topics())

[(0,
  '0.052*"work" + 0.049*"form" + 0.041*"contract" + 0.041*"ucsd" + '
  '0.030*"employee" + 0.027*"sign" + 0.024*"year" + 0.023*"agreement" + '
  '0.022*"payment_compass" + 0.019*"end"'),
 (1,
  '0.068*"submit" + 0.060*"request" + 0.058*"service" + 0.038*"pay" + '
  '0.028*"irequ" + 0.027*"company" + 0.026*"good" + 0.026*"already" + '
  '0.023*"event" + 0.019*"way"'),
 (2,
  '0.054*"approval" + 0.028*"unit" + 0.027*"currently" + 0.025*"comment" + '
  '0.025*"pende" + 0.023*"approver" + 0.023*"reply" + 0.019*"document" + '
  '0.016*"move" + 0.016*"believe"'),
 (3,
  '0.085*"line" + 0.082*"revision" + 0.066*"sure" + 0.056*"search" + '
  '0.036*"longer" + 0.033*"total" + 0.025*"accept" + 0.019*"field" + '
  '0.019*"period" + 0.017*"explain"'),
 (4,
  '0.059*"type" + 0.055*"amount" + 0.037*"buy" + 0.034*"invite" + '
  '0.031*"website" + 0.025*"http" + 0.024*"incorrect" + 0.021*"solution" + '
  '0.018*"good_morne" + 0.018*"meal"'),
 (5,
  '0.080*"message" + 0.065*"account" + 0.063*"busi

In [78]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.280922 -0.083117       1        1  23.960609
9     -0.265652  0.058976       2        1  20.044926
7     -0.181981  0.155284       3        1  14.722802
1      0.037686 -0.333875       4        1  11.691861
0      0.103393 -0.015662       5        1   7.384610
5      0.119361  0.060901       6        1   6.799934
6      0.074084 -0.149222       7        1   6.278213
2      0.098039  0.137364       8        1   3.902900
4      0.142956  0.081343       9        1   2.935870
3      0.153037  0.088008      10        1   2.278278, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
55    Default  2325.000000         order  2325.000000  30.0000  30.0000
27    Default  1307.000000          need  1307.000000  29.0000  29.0000
72    Default  1069.000000          send  1069.000000  28.0000  28.0000
217   Default   607.000000       invoice   607.000000  27.0000  27.0000
36    Default   999.000000       receive   999.000000  26.0000  26.0000
41    Default   729.000000        submit   729.000000  25.0000  25.0000
10    Default   954.000000         email   954.000000  24.0000  24.0000
256   Default   870.000000      supplier   870.000000  23.0000  23.0000
203   Default   495.000000       message   495.000000  22.0000  22.0000
57    Default   680.000000           try   680.000000  21.0000  21.0000
153   Default   738.000000       service   738.000000  20.0000  20.0000
37    Default   668.000000       request   668.000000  19.0000  19.0000
138   Default   810.000000   requisition   810.000000  18.0000  18.0000
131   Default   660.000000         index   660.000000  17.0000  17.0000
78    Default   447.000000       account   447.000000  16.0000  16.0000
126   Default   557.000000           add   557.000000  15.0000  15.0000
45    Default   681.000000           use   681.000000  14.0000  14.0000
320   Default   398.000000      business   398.000000  13.0000  13.0000
51    Default   362.000000         error   362.000000  12.0000  12.0000
159   Default   675.000000           ask   675.000000  11.0000  11.0000
133   Default  1526.000000   marketplace  1526.000000  10.0000  10.0000
102   Default   656.000000        vendor   656.000000   9.0000   9.0000
165   Default   638.000000          item   638.000000   8.0000   8.0000
202   Default   462.000000        change   462.000000   7.0000   7.0000
63    Default   552.000000       contact   552.000000   6.0000   6.0000
15    Default   332.000000          form   332.000000   5.0000   5.0000
111   Default   311.000000       payment   311.000000   4.0000   4.0000
47    Default   398.000000          work   398.000000   3.0000   3.0000
21    Default   606.000000          know   606.000000   2.0000   2.0000
147   Default   601.000000          help   601.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
88    Topic10   174.849228          line   175.779022   3.7764  -2.4686
254   Topic10   168.991409      revision   169.921204   3.7763  -2.5026
347   Topic10   136.916718          sure   137.846558   3.7750  -2.7131
760   Topic10   116.369324        search   117.299072   3.7738  -2.8757
1003  Topic10    73.633949        longer    74.563744   3.7692  -3.3334
827   Topic10    67.698189         total    68.628006   3.7681  -3.4174
1027  Topic10    50.904198        accept    51.834076   3.7636  -3.7025
707   Topic10    39.155010         field    40.084759   3.7583  -3.9650
703   Topic10    38.205563        period    39.135620   3.7577  -3.9895
756   Topic10    34.635727       explain    35.565662   3.7553  -4.0876
1196  Topic10    33.105701           fee    34.035488   3.7541  -4.1328
773   Topic10    32.184879      specific    33.114799   3.7533  -4.1610
708   Topic10    31.173456   performance    32.103577   3.7524  -4.19

In [92]:
#input term. output is [topic number, probability in topic]
pprint(lda_model.get_term_topics("invoice"))

[(6, 0.10647015)]


In [101]:
#Get the prob of each topic for each document.
p_doc_topic = lda_model.get_document_topics(corpus)

In [109]:
#use .get_document_topics methods to see topic chosen for doument & print original document/case.
print(p_doc_topic[2])
print(df_txt[2])

[(0, 0.048439912), (1, 0.08056676), (2, 0.123441644), (3, 0.0179537), (4, 0.01620218), (5, 0.03928752), (6, 0.03540441), (7, 0.13165464), (8, 0.29698944), (9, 0.21005978)]
PO # 91785100
Req # XXXXXXXXX

According to Dell this order was cancelled. I didn't receive any notice of that. Could you please check.
Thank You,
Jim Beadle
UCSD Medicine
(858) 699-8223
['po', 'req', 'xxxxxxxxx', 'according', 'dell', 'order', 'cancelled', 'receive', 'notice', 'could', 'please', 'check', 'jim', 'beadle', 'ucsd', 'medicine']


In [115]:
pprint(lda_model.show_topic(0))

[('work', 0.052473124),
 ('form', 0.04949468),
 ('contract', 0.041153282),
 ('ucsd', 0.04097518),
 ('employee', 0.029979082),
 ('sign', 0.027310072),
 ('year', 0.023953043),
 ('agreement', 0.022606347),
 ('payment_compass', 0.021781508),
 ('end', 0.018525349)]
